In [1]:
import json
import numpy as np
import sys
sys.path.append('../src')
import cpu

with open('C:/Users/David/Documents/Coding/daveNES/tests/ProcessorTests-main/nes6502/v1/61.json') as f:
    test = json.load(f)

pygame 2.3.0 (SDL 2.24.2, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
test[0]

{'name': '61 90 d3',
 'initial': {'pc': 24473,
  's': 48,
  'a': 201,
  'x': 139,
  'y': 31,
  'p': 44,
  'ram': [[24473, 97],
   [24474, 144],
   [24475, 211],
   [144, 105],
   [27, 231],
   [28, 120],
   [30951, 128]]},
 'final': {'pc': 24475,
  's': 48,
  'a': 73,
  'x': 139,
  'y': 31,
  'p': 109,
  'ram': [[27, 231],
   [28, 120],
   [144, 105],
   [24473, 97],
   [24474, 144],
   [24475, 211],
   [30951, 128]]},
 'cycles': [[24473, 97, 'read'],
  [24474, 144, 'read'],
  [144, 105, 'read'],
  [27, 231, 'read'],
  [28, 120, 'read'],
  [30951, 128, 'read']]}

In [3]:
def init_daveNES(test):
    daveNES = cpu.MOS6502(debug = False)
    daveNES.initialise_RAM()

    # Load test program
    for val in test['initial']['ram']:
        print(f'{val[0]}, {val[1]}')
        daveNES.ram.memory[val[0]] = np.uint8(val[1])
    daveNES.r_program_counter = test['initial']['pc']
    daveNES.r_stack_pointer = test['initial']['s']
    daveNES.r_accumulator = test['initial']['a']
    daveNES.r_index_X = test['initial']['x']
    daveNES.r_index_Y = test['initial']['y']
    daveNES.value_to_status(test['initial']['p'])

    return daveNES

In [4]:
test[0]['initial']['ram']

[[24473, 97],
 [24474, 144],
 [24475, 211],
 [144, 105],
 [27, 231],
 [28, 120],
 [30951, 128]]

In [5]:
daveNES = init_daveNES(test[0])

24473, 97
24474, 144
24475, 211
144, 105
27, 231
28, 120
30951, 128


In [57]:
# Indirect X addressing mode
daveNES.r_program_counter = test[0]['initial']['pc']
base = daveNES.ram.read(daveNES.r_program_counter)
daveNES.r_program_counter += 1
ptr = np.uint8(base) + np.uint8(daveNES.r_index_X) # Wrapping Add (may throw overflow exception)
print(f'ptr: {ptr}')
lo = daveNES.ram.read(ptr.astype(np.uint16))
hi = daveNES.ram.read(
    (ptr + np.uint8(1)).astype(np.uint16)
)  # Wrapping Add (may throw overflow exception)
addr = np.uint16(hi) << 8 | np.uint16(lo)

print(f'addr: {addr}')

#addr = np.uint16(30951) # This seems to be the issue, we want to point to 30951 as a np.uint16
value = daveNES.ram.read(addr)

a = np.uint16(daveNES.r_accumulator)
m = np.uint16(value)
c = np.uint16(daveNES.r_status["flag_C"])

result = a + m + c

print(np.uint8(result))

ptr: 236
addr: 0
201


In [100]:
pc = test[0]['initial']['pc'] + 1
base = daveNES.ram.read(pc)
pc += 1
print(f'base: {base}')
ptr = base + np.uint8(daveNES.r_index_X)
print(f'ptr: {ptr}')
#daveNES.ram.read(base + daveNES.r_index_X + np.uint8(1))

# target_number is 30951

base: 144
ptr: 27


C:\Users\David\AppData\Local\Temp\ipykernel_19140\3472765775.py:5: RuntimeWarning: overflow encountered in ubyte_scalars
  ptr = base + np.uint8(daveNES.r_index_X)


In [101]:
daveNES.ram.read(np.uint16(ptr + 1) & 0x00FF) << 8 | daveNES.ram.read(np.uint16(ptr) & 0x00FF)

30951

In [102]:
daveNES.ram.read(np.uint16(ptr + np.uint8(1))) &0x00FF << 8

0

In [95]:
daveNES.ram.read(np.uint16(ptr) & 0x00FF)

231

In [ ]:
print(f'{test["final"]["pc"]} : {daveNES.r_program_counter}'
    assert test['final']['s'] == daveNES.r_stack_pointer
    assert test['final']['a'] == daveNES.r_accumulator
    assert test['final']['x'] == daveNES.r_index_X
    assert test['final']['y'] == daveNES.r_index_Y